Problem 2

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [3]:
bikeshare_dock_locations = pd.read_csv('bikeshare_dock_locations.csv')
bikeshare_customer_locations = pd.read_csv('bikeshare_customer_locations.csv')
bikeshare_time = pd.read_csv('bikeshare_time.csv', header=None).values

In [4]:
bikeshare_customer_locations.head()

,customer,longitude,latitude,walktime
0,1,-118.464515,34.036088,300
1,2,-118.461880,34.033794,120
2,3,-118.461481,34.033850,480
3,4,-118.486521,34.024456,360
4,5,-118.470259,34.039326,600


In [5]:
bikeshare_dock_locations.head()

,candidate,longitude,latitude,capacity
0,1,-118.494089,34.019297,60
1,2,-118.479515,34.012164,70
2,3,-118.481021,34.029711,50
3,4,-118.461908,34.017341,60
4,5,-118.494246,34.037268,80


In [6]:
bikeshare_time

array([[2946, 2987, 1627, ..., 2245, 1278, 2044],
       [1320, 2230,  207, ..., 1294, 3257,  552],
       [ 784, 2154, 3041, ..., 1540, 1008,  480],
       ...,
       [ 755, 2252, 1985, ..., 1765, 1721, 2486],
       [ 840, 2678,  255, ...,  291, 3512,  369],
       [1173, 1567, 3012, ..., 2523, 2370, 2366]])

Part 1a

\begin{align}

\text{minimize} \quad & \frac{1}{m} \sum_{j=1}^m \sum_{i=1}^n t_{i,j} y_{i,j} \;\;\;\; \text{[Objective Function]}\\
\text{subject to} \quad & \sum_{i=1}^n y_{i,j} = 1, \quad \forall j \in \{1,\dots,m\}, \;\;\;\; \text{[Serve each Customer Constraint]}\\
& y_{i,j} \leq x_i, \quad \forall i \in \{1,\dots,n\}, \ j \in \{1,\dots,m\}, \;\;\;\; \text{[Assignment Constraint]}\\
& \sum_{i=1}^n x_i \leq 10, \;\;\;\; \text{[10 Stations Constraint]}\\
& x_{i} \in \{0,1\}, \quad \forall i \in \{1,\dots,n\}, \;\;\;\; \text{[Binary Constraint]}\\
& y_{i,j} \in \{0,1\}, \quad \forall i \in \{1,\dots,n\}, \ j \in \{1,\dots,m\}. \;\;\;\; \text{[Binary Constraint]}

\end{align}

where decision variables are given as follows:

$t_{i,j}$ is the walktime from customer $j$ to docking station $i$ in seconds

$y_{i,j}$ is a binary variable to decide if customer $j$ is assigned to docking station $i$ or not

$x_i$ is a binary variable to decide if docking station $i$ is open or not

to minimize average walktime for all customers.

In [8]:
m = bikeshare_customer_locations.shape[0]
n = bikeshare_dock_locations.shape[0]

In [7]:
# Part 1b

m = bikeshare_customer_locations.shape[0]
n = bikeshare_dock_locations.shape[0]

from gurobipy import *

m1 = Model()
m1.Params.LogToConsole = 0

x = m1.addVars(n, vtype = GRB.BINARY)
y = m1.addVars(n, m, vtype = GRB.BINARY)

for j in range(m):
    m1.addConstr(sum(y[i,j] for i in range(n)) == 1)
    for i in range(n):
        m1.addConstr(y[i,j] <= x[i])

m1.addConstr( sum(x[i] for i in range(n)) <= 10)
m1.setObjective(1.0/m * sum( bikeshare_time[i,j] * y[i,j] for i in range(n) for j in range(m)), GRB.MINIMIZE)

m1.update()
m1.optimize()

print('Optimal Objective Value:', m1.objVal)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-13

Interrupt request received
Optimal Objective Value: 492.48714285714243


In [ ]:
# Part 1c

locs = [i+1 for i in range(n) if x[i].x > 0.5]
print(locs)

[4, 8, 9, 11, 19, 24, 25, 27, 28, 38]


Part 2a

\begin{align}

\text{minimize} \quad & r \;\;\;\; \text{[Objective Function]}\\
\text{subject to} \quad & r \ge \sum_{i=1}^n t_{i,j} y_{i, j}, \quad \forall j \in \{1,\dots,m\}, \;\;\;\; \text{[Maximum Walktime Constraint]}\\
& y_{i,j} \leq x_i, \quad \forall i \in \{1,\dots,n\}, \ j \in \{1,\dots,m\}, \;\;\;\; \text{[Assignment Constraint]}\\
& \sum_{i=1}^n x_i = 10, \;\;\;\; \text{[10 Stations Constraint]}\\
& x_{i} \in \{0,1\}, \quad \forall i \in \{1,\dots,n\}, \;\;\;\; \text{[Binary Constraint]}\\
& y_{i,j} \in \{0,1\}, \quad \forall i \in \{1,\dots,n\}, \ j \in \{1,\dots,m\}. \;\;\;\; \text{[Binary Constraint]}

\end{align}

where decision variables are given as follows:

$t_{i,j}$ is the walktime from customer $j$ to docking station $i$ in seconds

$y_{i,j}$ is a binary variable to decide if customer $j$ is assigned to docking station $i$ or not

$x_i$ is a binary variable to decide if docking station $i$ is open or not

$r$ is the maximum walktime

to minimize maximum walktime for all customers.

In [ ]:
# Part 2b

m2 = Model()
m2.Params.LogToConsole = 0

x = m2.addVars(n, vtype = GRB.BINARY)
y = m2.addVars(n,m, vtype = GRB.BINARY)
r = m2.addVar()


# Add the constraints:
for j in range(m):
    m2.addConstr(sum(y[i,j] for i in range(n)) == 1) # each customer must be served
    for i in range(n):
        m2.addConstr(y[i,j] <= x[i]) # a customer j is served by location i only if location i is opened
    
    m2.addConstr(r >= sum( bikeshare_time[i,j] * y[i,j] for i in range(n)))

# Add the constraint that we do not open more than ten docking stations
m2.addConstr( sum(x[i] for i in range(n)) <= 10)

# Set the objective (minimize average walking distance for all customers)
m2.setObjective(r, GRB.MINIMIZE)

# Solve the problem!
m2.update()
m2.optimize()

print('Optimal Objective Value:', m2.objVal)

Optimal Objective Value: 1268.0


In [ ]:
# Part 2c

locs = [i+1 for i in range(n) if x[i].x > 0.5]
print(locs)

[5, 13, 15, 20, 22, 25, 26, 28, 30, 42]


Part 3a

\begin{align}

\text{maximize} \quad & \sum_{j=1}^m z_j \;\;\;\; \text{[Objective Function]}\\
\text{subject to} \quad & z_j \le \sum_{i=1}^n a_{i,j} x_i, \quad \forall j \in \{1,\dots,m\}, \;\;\;\; \text{[Coverage Constraint]}\\
& \sum_{i=1}^n x_i = 10, \;\;\;\; \text{[10 Stations Constraint]}\\
& x_{i} \in \{0,1\}, \quad \forall i \in \{1,\dots,n\}, \;\;\;\; \text{[Binary Constraint]}\\
& z_j \in \{0,1\}, \quad \forall \ j \in \{1,\dots,m\}. \;\;\;\; \text{[Binary Constraint]}

\end{align}

where decision variables are given as follows:

$a_{i,j}$ is the coverage parameter obtained from problem data whether docking station $i$ covers customer $j$ or not

$z_j$ is a binary variable to decide if we serve customer $j$ or not

$x_i$ is a binary variable to decide if docking station $i$ is open or not

to maximize coverage for all customers.

In [ ]:
# Part 3b

coverage_time = 600
a_mat = np.asarray([[int(bikeshare_time[i,j] < coverage_time) for j in range(m)] for i in range(n)])

# Create the Gurobi model. 
m3 = Model()
m3.Params.LogToConsole = 0

x = m3.addVars(n, vtype = GRB.BINARY)
z = m3.addVars(m, vtype = GRB.BINARY)

# Create coverage constraints: 
# if z[j] is 1 then for at least one i that covers j, x[i] must be equal to 1.
for j in range(m):
    m3.addConstr(z[j] <= sum(a_mat[i, j] * x[i] for i in range(n)))

m3.addConstr(sum(x[i] for i in range(n)) == 10) 


m3.setObjective( sum(z[j] for j in range(m) ), GRB.MAXIMIZE)

m3.update()
m3.optimize()

print('Optimal Objective Value:', m3.objVal)

Optimal Objective Value: 496.0


In [ ]:
# Part 3c

locs = [i+1 for i in range(n) if x[i].x > 0.5]
print(locs)

[8, 9, 12, 15, 19, 24, 25, 28, 34, 37]


In [ ]:
a_mat.shape

(42, 700)

In [15]:
walktime = bikeshare_customer_locations['walktime'].values
closest_mat = bikeshare_time[i, :] * x[i] for i in range(n)
closest_mat = np.asarray([bikeshare_time[:, j].argmin() for j in range(m)])
closest_mat = np.asarray([int(bikeshare_time[closest_mat[j], j] < walktime[j]) for j in range(m)])
closest_mat.sum()

485

In [17]:
# From further customer surveys, the company has determined that each customer will in general use the dock location that is 
# nearest to him/her (requiring the least amount of time to walk to). Each potential customer has indicated to the company the 
# most amount of time that he/she would be willing to walk to reach a dock location. This time varies between 2 and 10 minutes 
# for each customer. The file bikeshare-customer-locations-v2.csv contains the maximum amount of time that each customer has 
# indicated that they would walk (walktime). If the closest dock location is farther in time than this maximum amount, the customer 
# will give up and not retrieve a bike.
# An additional concern that the company has recognized is that the candidate sites for the docking stations are limited in capacity. 
# The file bikeshare-dock-locations-v2.csv includes how many bikes will be stored at each candidate location (capacity). If the number 
# of customers who choose to walk to a particular location exceeds the capacity of that location, the customers in excess of the capacity 
# will not be able to obtain a bike.
# The company wishes to minimize an objective that captures the sum of two quantities:
# • F1: the number of customers who do not obtain a bike (either because a bike is unavailable at their closest dock location, or because 
# the closest dock location was too far); and
# • F2: the number of bikes at an open docking station that are unused by any customer.
# The company determines that F1 is twice as important as F2, and hence wishes to minimize 2F1 + F2.
# a) Write down an abstract formulation of the problem as an integer programming problem. Define the decision variables, constraints and 
# objective function, and any parameters that your formu- lation requires. Explain why your formulation is correct. (Do not formulate any 
# constraints with actual numbers, with the exception of the requirement that 10 docking stations be selected.)
# Formulate and solve your problem in Python using Gurobi.

walktime = bikeshare_customer_locations['walktime'].values
capacity = bikeshare_dock_locations['capacity'].values

# Create the Gurobi model.
m4 = Model()
m4.Params.LogToConsole = 0

x = m4.addVars(n, vtype = GRB.BINARY) # location i is opened
z = m4.addVars(m, vtype = GRB.BINARY) # customer j is served



for j in range(m):
    m3.addConstr(z[j] <= sum(a_mat[i, j] * x[i] * closest_mat for i in range(n)))

# Create the constraints
for j in range(m):
    # m4.addConstr( sum(y[i,j] for i in range(n)) == 1) # each customer must be served
    for i in range(n):
        m4.addConstr( y[i,j] <= x[i] ) # a customer j is served by location i only if location i is opened
        m4.addConstr( a_mat[i,j] * x[i] <= z[i] ) # if a customer j is served by location i, then location i must be opened

m4.addConstr( sum(x[i] for i in range(n)) == 10) # we must open 10 locations



# Set the objective
m4.setObjective( 2 * ((m - sum(z[j] for j in range(m)) + sum(capacity[i] - x[i]* for i in range(n))) + sum( x[i] for i in range(n) ), GRB.MINIMIZE) # minimize the number of customers who do not obtain a bike and the number of unused bikes

m4.update()
m4.optimize()

m4.ObjVal

30.0

In [18]:
locs = [i+1 for i in range(n) if x[i].x > 0.5]
print(locs)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [19]:
# how many customers do not obtain a bike
sum(y[i,j].x for i in range(n) for j in range(m) )

700.0

In [20]:
a_mat

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 1, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]])